<a href="https://colab.research.google.com/github/Johnny880724/AFSA_material/blob/main/fish_on_real_crystal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Step 1: Mount Google Drive to access your files
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import necessary libraries
import numpy as np

# Step 3: Define the path to your .txt file
# You can find the path after mounting your Google Drive.
# Make sure to adjust the path based on where your file is located in Drive.
file_path = '/content/drive/MyDrive/crystal data/5A_parsed_file.txt'

# Step 4: Load the .txt file into a NumPy array
# Assuming the file is space-separated, but adjust delimiter if needed
matrix = np.loadtxt(file_path)

coordinates = matrix[:, 1:]  # Extract all rows except the first
bounds = np.array([[np.min(coordinates[:, i]), np.max(coordinates[:, i])] for i in range(coordinates.shape[1])])

# Step 6: Display the calculated bounds
print("Calculated Bounds for Each Dimension:")
print(bounds)

# Step 5: Display the matrix

print(np.min(matrix[:, 0]))
print("Matrix Loaded from File:")
print(coordinates[1])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Calculated Bounds for Each Dimension:
[[  1.621  84.27 ]
 [-20.     17.93 ]
 [-41.37   42.61 ]
 [  0.      0.   ]
 [  3.215  90.53 ]
 [-23.85  141.7  ]
 [  0.      0.   ]
 [  0.      0.   ]
 [  3.704  66.77 ]]
-153.5799
Matrix Loaded from File:
[ 4.961e+00  6.140e-16 -3.787e+00  0.000e+00  1.864e+01  4.770e-15
  0.000e+00  0.000e+00  6.941e+00]


MultiGaussian fitting

In [40]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def fit_gaussian_process(input_array):
    """
    Fit a Gaussian Process model to the provided n-dimensional array.

    Args:
    - input_array (np.ndarray): The n-dimensional array where the first row contains energy (z-values)
                                and the subsequent rows represent the coordinates in n-dimensional space.

    Returns:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    """
    # Separate the energy (first row) and location data (other rows)
    energy_values = input_array[0, :]  # Energy values (z)
    location_data = input_array[1:, :].T  # Location data (x, y, ...)

    # Define the kernel: we use a product of constant kernel and RBF kernel (Gaussian kernel)
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))  # Constant * RBF kernel

    # Initialize the GaussianProcessRegressor with the kernel
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

    # Fit the GP model to the data
    gp.fit(location_data, energy_values)

    return gp

def predict_energy(gp, location):
    """
    Predict the energy at a given location using the fitted Gaussian Process model.

    Args:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    - location (np.ndarray): The location for which to predict the energy.

    Returns:
    - energy (float): The predicted energy at the given location.
    """
    # Ensure the location is in the right shape (1 x n_dim)
    location = np.array(location).reshape(1, -1)

    # Use the Gaussian Process model to predict the energy at the specified location
    energy, _ = gp.predict(location, return_std=True)

    return energy[0]  # Return the predicted energy value

# Example Usage:
# Input: An example n-dimensional array where the first row is energy, and others are coordinates.
input_array = np.array([
    [1.0, 2.0, 3.0, 4.0],  # Energy (z-values)
    [0.0, 1.0, 2.0, 3.0],  # Location 1 (x)
    [1.0, 1.5, 2.0, 2.5],  # Location 2 (y)
])

# Fit the Gaussian Process model to the input data
gp_model = fit_gaussian_process(matrix.T)

# Predict the energy at a new location (e.g., at location [2.5, 2.0])
new_location = [0,1,0,0,0,1,1,0,1]

predicted_energy = predict_energy(gp_model, new_location)

# Output the predicted energy
print(f"Predicted energy at location {new_location}: {predicted_energy}")


Predicted energy at location [0, 1, 0, 0, 0, 1, 1, 0, 1]: -0.0004471366392596217


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


Fish sworm


In [41]:
import numpy as np

class ArtificialFishSwarm:
    def __init__(self, objective_function, n_dim, bounds, population_size=50, max_iter=100, step_size=0.1, visual_range=0.5):
        self.objective_function = objective_function
        self.n_dim = n_dim
        self.bounds = bounds  # Bounds shape: (n_dim, 2)
        self.population_size = population_size
        self.max_iter = max_iter
        self.step_size = step_size
        self.visual_range = visual_range

        # Initialize population (randomly within bounds)
        self.population = np.random.uniform(
            self.bounds[:, 0], self.bounds[:, 1], (self.population_size, self.n_dim)
        )
        # Fitness values for each fish
        self.fitness = np.array([self.objective_function(ind) for ind in self.population])

        # Global best solution
        self.best_fish = self.population[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)

    def move_fish(self, fish, target):
        """
        Move a fish towards a target fish within the step size.
        """
        direction = target - fish
        distance = np.linalg.norm(direction)

        if distance > 0:
            direction = direction / distance  # Normalize direction

        # Move the fish in the direction of the target
        new_position = fish + self.step_size * direction

        # Clip the new position to remain within bounds
        return np.clip(new_position, self.bounds[:, 0], self.bounds[:, 1])

    def search_neighborhood(self, fish):
        """
        Search the neighborhood of a fish for a better position.
        """
        best_fish_in_neighborhood = fish
        best_fitness_in_neighborhood = self.objective_function(fish)

        # Randomly explore the neighborhood
        for _ in range(10):  # Arbitrary number of random searches
            random_step = np.random.uniform(-self.visual_range, self.visual_range, self.n_dim)
            candidate_fish = fish + random_step
            candidate_fish = np.clip(candidate_fish, self.bounds[:, 0], self.bounds[:, 1])  # Clip within bounds
            candidate_fitness = self.objective_function(candidate_fish)

            if candidate_fitness < best_fitness_in_neighborhood:
                best_fish_in_neighborhood = candidate_fish
                best_fitness_in_neighborhood = candidate_fitness

        return best_fish_in_neighborhood

    def run(self):
        """
        Run the Artificial Fish Swarm Algorithm to optimize the objective function.
        """
        for iteration in range(self.max_iter):
            for i in range(self.population_size):
                fish = self.population[i]

                # Search the neighborhood for a better position
                best_fish_in_neighborhood = self.search_neighborhood(fish)

                # Move towards the best position in the neighborhood
                new_fish = self.move_fish(fish, best_fish_in_neighborhood)

                # Update fitness
                new_fitness = self.objective_function(new_fish)

                # If the new position is better, update the fish
                if new_fitness < self.fitness[i]:
                    self.population[i] = new_fish
                    self.fitness[i] = new_fitness

                    # Update global best solution
                    if new_fitness < self.best_fitness:
                        self.best_fish = new_fish
                        self.best_fitness = new_fitness

            # Print progress
            print(f"Iteration {iteration + 1}: Best Fitness = {self.best_fitness}")

        return self.best_fish, self.best_fitness


In [47]:
# Example objective function using GP
def gp_objective_function(location):
    return predict_energy(gp_model, location)

# Define the bounds for 9 dimensions (example: [0, 1] for each dimension)
n_dim = 9
#bounds = np.tile([0, 1], (n_dim, 1))

# Create the AFSA object
afsa = ArtificialFishSwarm(
    objective_function=gp_objective_function,
    n_dim=n_dim,
    bounds=bounds,
    population_size=50,
    max_iter=200,
    step_size=0.5,
    visual_range=0.5
)

# Run the AFSA algorithm
best_solution, best_fitness = afsa.run()

# Output the results
print("\nGlobal Minimum Location:", best_solution)
print("Global Minimum Energy:", best_fitness)


Iteration 1: Best Fitness = -0.14823308105884875
Iteration 2: Best Fitness = -0.2145746372802058
Iteration 3: Best Fitness = -0.3050237725913518
Iteration 4: Best Fitness = -0.3606916152828353
Iteration 5: Best Fitness = -0.5577292859574887
Iteration 6: Best Fitness = -0.8437381853868394
Iteration 7: Best Fitness = -1.084609039534281
Iteration 8: Best Fitness = -1.5659630835591931
Iteration 9: Best Fitness = -1.9648037852253137
Iteration 10: Best Fitness = -2.7919385639461543
Iteration 11: Best Fitness = -4.217678431867171
Iteration 12: Best Fitness = -5.926078030408515
Iteration 13: Best Fitness = -8.348835037506236
Iteration 14: Best Fitness = -10.358382589319053
Iteration 15: Best Fitness = -14.424932344989243
Iteration 16: Best Fitness = -20.167303809501096
Iteration 17: Best Fitness = -22.873070827001854
Iteration 18: Best Fitness = -26.518359328105518
Iteration 19: Best Fitness = -28.272059571964316
Iteration 20: Best Fitness = -36.23330332993107
Iteration 21: Best Fitness = -39.